# Introduction
Building on the model visualization of Mol et al., the acetone production pathway is added 

In [3]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite
from os.path import dirname, join, pardir

data_dir = "/Users/phivri/Documents/GitHub/MoGeoModel"

In [2]:
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

solution = model.optimize()
Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 10.436720375457611, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, …

In [ ]:
model.reactions.get_by_id("EX_Biomass_e")

In [8]:
#Desired ratio of max growth, chosen from PPP:
ratio = 0.5

XFlux = model.slim_optimize()*ratio

with model:
    model.reactions.get_by_id("EX_Biomass_e").bounds = XFlux, XFlux
    model.objective = "EX_act_e"
    PFlux = model.slim_optimize()

set_flux = model.problem.Constraint(
    XFlux * model.reactions.EX_act_e.flux_expression - PFlux * model.reactions.EX_Biomass_e.flux_expression,
    lb=0,
    ub=0)
model.add_cons_vars(set_flux)

solution = model.optimize()

Builder(model = model, map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

0.032978270242252763
0.990463355198281


Builder(reaction_data={'IDPh': 8.237433559589501, 'CAT': -5.141076088614215e-09, 'PDHam1hi': 0.0, 'HYDA': 0.0,…

In [ ]:
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

set_flux = model.problem.Constraint(
    model.reactions.EX_act_e.flux_expression - model.reactions.EX_Biomass_e.flux_expression,
    lb=0,
    ub=0)
model.add_cons_vars(set_flux)

model.objective = "EX_Biomass_e"
model.summary()

In [ ]:
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

with model:
    model.objective = "EX_Biomass_e"
    print(model.summary())

with model:
    model.objective = "EX_act_e"
    print(model.summary())

In [ ]:
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

In [ ]:
acetone_exchange = model.reactions.get_by_id("EX_act_e")
biomass = model.reactions.get_by_id("EX_Biomass_e")

In [ ]:
biomass80 = model.problem.Constraint(
    0.2 * acetone_exchange.flux_expression + 0.08 * biomass.flux_expression,
    lb=1,
    ub=1,
)

In [ ]:
at_least_biomass80 = model.problem.Constraint(
    0.8 * acetone_exchange.flux_expression - 0.2 * biomass.flux_expression,
    lb=-1000,
    ub=0,
)
model.add_cons_vars(at_least_biomass80)

print(f"Growth after custom constraint: {model.slim_optimize()}")

changed_objective = model.problem.Objective(
    0.2 * acetone_exchange.flux_expression + 0.8 * biomass.flux_expression,
    direction="max",
)

model.objective = changed_objective
print(f"Changed objective: {model.slim_optimize()}")
print(f"Gluconate exchange: {acetone_exchange.flux}, Biomass value: {biomass.flux}")

solution = model.optimize()

Builder(model = model, map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
model.summary()

In [ ]:
#with model:
#    model.add_cons_vars(biomass80)
#    print(model.summary())

In [ ]:
at_least_biomass80 = model.problem.Constraint(
    0.5 * acetone_exchange.flux_expression - 0.5 * biomass.flux_expression,
    lb=-1000,
    ub=0,
)

changed_objective = model.problem.Objective(
    0.5 * acetone_exchange.flux_expression + 0.5 * biomass.flux_expression,
    direction="max",
)

In [ ]:
#with model:
model.add_cons_vars(at_least_biomass80)
model.objective = changed_objective
#print(model.summary())
solution = model.optimize()
Builder(model = model, map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

Y_XP_max = solution.fluxes.EX_Biomass_e/solution.fluxes.EX_acta_e

print(f"The maximum yield on acetate is {Y_XP_max}")

ratio = 0.8

print(f"The acetone producing strain is expected to grow to a minimum of {ratio*100}% of its maximum Biomass yield")

In [ ]:
MaxGrowth = model.problem.Constraint(
    model.reactions.EX_Biomass_e.flux_expression - Y_XP_max * ratio * model.reactions.EX_act_e.flux_expression,
    lb=0,
    ub=1000)
model.add_cons_vars(MaxGrowth)

model.objective = 'EX_act_e'

solution = model.optimize()

model.summary()

In [ ]:
Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
Builder(model = model, map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())